# 0. 환경 설정 및 유틸 함수

In [1]:
# ENV SETTING
import os, json, glob, shutil, random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import pandas as pd
import torch

# 랜덤 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print("Seed fixed.")

# 디바이스 자동 선택
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("사용 디바이스:", device)

Seed fixed.
사용 디바이스: mps


# 1. 경로 설정

In [2]:
BASE = "/Users/apple/Downloads/codeit/project/ai06-level1-project"

TRAIN_IMG_DIR = f"{BASE}/train_images"
TRAIN_ANN_DIR = f"{BASE}/train_annotations"
TEST_IMG_DIR  = f"{BASE}/test_images"

YOLO_DIR = f"{BASE}/yolo_multiclass"

### YOLO 훈련용 데이터 구조 생성

In [3]:
os.makedirs(f"{YOLO_DIR}/images/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/images/val", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/val", exist_ok=True)

# 2. JSON 스캔 후 category_id 수집 + 매핑 만들기

In [4]:
json_paths = glob.glob(f"{TRAIN_ANN_DIR}/**/*.json", recursive=True)

category_ids = set()

file_to_json = {}

for jp in json_paths:
    try:
        with open(jp, "r") as f:
            data = json.load(f)

        img_name = data["images"][0]["file_name"]
        file_to_json[img_name] = jp

        anns = data["annotations"]
        for ann in anns:
            category_ids.add(ann["category_id"])

    except:
        print("⚠ JSON 파싱 실패:", jp)

sorted_ids = sorted(list(category_ids))
print("총 고유 category_id 개수:", len(sorted_ids))

# 매핑 생성
cat2idx = {cid: i for i, cid in enumerate(sorted_ids)}
idx2cat = {i: cid for i, cid in enumerate(sorted_ids)}

# 저장
with open(f"{BASE}/category_mapping.json", "w") as f:
    json.dump({"cat2idx": cat2idx, "idx2cat": idx2cat}, f, indent=4)

print("category_mapping.json 저장 완료!")

총 고유 category_id 개수: 56
category_mapping.json 저장 완료!


# 3. JSON → YOLO 라벨 변환 + train/val split

In [5]:
image_files = sorted([f for f in os.listdir(TRAIN_IMG_DIR) if f.endswith(".png")])

random.shuffle(image_files)
split = int(len(image_files) * 0.9)

train_imgs = image_files[:split]
val_imgs   = image_files[split:]

def convert_bbox(x, y, w, h, img_w, img_h):
    cx = (x + w/2) / img_w
    cy = (y + h/2) / img_h
    w  = w / img_w
    h  = h / img_h
    return cx, cy, w, h


for img_name in tqdm(image_files):
    src = f"{TRAIN_IMG_DIR}/{img_name}"

    if img_name in train_imgs:
        img_dst = f"{YOLO_DIR}/images/train/{img_name}"
        label_dst = f"{YOLO_DIR}/labels/train/{img_name.replace('.png','.txt')}"
    else:
        img_dst = f"{YOLO_DIR}/images/val/{img_name}"
        label_dst = f"{YOLO_DIR}/labels/val/{img_name.replace('.png','.txt')}"

    shutil.copy(src, img_dst)

    if img_name not in file_to_json:
        continue

    json_path = file_to_json[img_name]
    with open(json_path,"r") as f:
        data = json.load(f)

    img_w = data["images"][0]["width"]
    img_h = data["images"][0]["height"]

    with open(label_dst, "w") as lf:
        for ann in data["annotations"]:
            x, y, w, h = ann["bbox"]
            cid = ann["category_id"]
            cls = cat2idx[cid]

            cx, cy, ww, hh = convert_bbox(x, y, w, h, img_w, img_h)

            lf.write(f"{cls} {cx} {cy} {ww} {hh}\n")

100%|██████████| 651/651 [00:01<00:00, 382.43it/s]


# 4. YOLO dataset.yaml 생성

In [6]:
yaml_content = f"""
path: {YOLO_DIR}
train: images/train
val: images/val

nc: {len(sorted_ids)}
names: {sorted_ids}
"""

with open(f"{YOLO_DIR}/dataset.yaml", "w") as f:
    f.write(yaml_content)

print("dataset.yaml 생성 완료!")

dataset.yaml 생성 완료!


# 5. YOLO 멀티클래스 학습

In [7]:
model = YOLO("yolov8m.pt")

model.train(
    data=f"{YOLO_DIR}/dataset.yaml",
    epochs=20,
    imgsz=640,
    batch=16,
    device=0 if device=="cuda" else "cpu",
    name="multiclass_pill",
)

Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/apple/Downloads/codeit/project/ai06-level1-project/yolo_multiclass/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=multiclass_pill, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, p

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 2,  4,  6, 12, 13, 18, 20, 23, 25, 28, 48, 52, 55])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x153100970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045, 

# 6. Kaggle 제출용 inference

In [8]:
# mapping 로드
with open(f"{BASE}/category_mapping.json") as f:
    mp = json.load(f)
idx2cat = {int(k): v for k, v in mp["idx2cat"].items()}

model = YOLO(f"{BASE}/runs/detect/multiclass_pill/weights/best.pt")

submission = []
ann_id = 1

for img_name in sorted(os.listdir(TEST_IMG_DIR)):
    if not img_name.endswith(".png"):
        continue

    img_id = int(img_name.replace(".png",""))
    img_path = f"{TEST_IMG_DIR}/{img_name}"

    results = model.predict(img_path)[0]

    for box in results.boxes:
        cls = int(box.cls)
        score = float(box.conf)

        orig_cid = idx2cat[cls]

        x1, y1, x2, y2 = box.xyxy[0]
        w = x2 - x1
        h = y2 - y1

        submission.append([
            ann_id, img_id, orig_cid,
            float(x1), float(y1), float(w), float(h), score
        ])
        ann_id += 1

df = pd.DataFrame(submission, columns=[
    "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
])

df.to_csv("kaggle_submission_ver2.csv", index=False)
print("kaggle_submission_ver2.csv 저장 완료!")


image 1/1 /Users/apple/Downloads/codeit/project/ai06-level1-project/test_images/1.png: 640x512 (no detections), 231.9ms
Speed: 2.1ms preprocess, 231.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /Users/apple/Downloads/codeit/project/ai06-level1-project/test_images/10.png: 640x512 (no detections), 231.1ms
Speed: 1.4ms preprocess, 231.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /Users/apple/Downloads/codeit/project/ai06-level1-project/test_images/100.png: 640x512 1 33008, 250.4ms
Speed: 1.7ms preprocess, 250.4ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /Users/apple/Downloads/codeit/project/ai06-level1-project/test_images/1003.png: 640x512 (no detections), 249.5ms
Speed: 1.9ms preprocess, 249.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /Users/apple/Downloads/codeit/project/ai06-level1-project/test_images/1004.png: 640x512 (no detections), 245.9ms
Sp